# Relations avec les institutions et spatialisation

In [ ]:
### Noter qu'il faudra avoir installé toutes les librairies nécessaires
# dans l'environnement conda utilisé pour exécuter ce carnet
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from geopandas.tools import sjoin
from geodatasets import get_path
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
from itables import init_notebook_mode, show
import numpy as np
import seaborn as sns
import json
import IPython

## Production des données

* Requête exécutée sur le serveur https://dbpedia.org/sparql 
* Période élargie jusqu'aux comtemporains-nes
* Propriétés prises en compte après inspection des propriétés de la population:
  * http://dbpedia.org/ontology/almaMater 
  * http://dbpedia.org/ontology/institution
* On ajoute les coordonnées géographiques
* Noter que la structure de la requête (sans clause OPTIONAL) exclut toutes les personnes pour lequelles une variable manque

```SPARQL
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT (?o1 AS ?subject_uri) ?subjectBirthDate ?birthPlace ?birthPoint ?r_property ?organisation ?orgPoint
# (COUNT(*) AS ?effectif) 
WHERE {
  SELECT DISTINCT ?o1 ?subjectBirthDate  ?birthPlace ?birthPoint ?r_property ?organisation ?orgPoint
  WHERE { 
    {
          {dbr:List_of_astronomers ?p ?o1.}
      UNION
          {dbr:List_of_astrologers ?p ?o1.}
      UNION
          {?o1 ?p dbr:Astrologer.}
      UNION
          {?o1 ?p dbr:Astronomer.}
      UNION
          {?o1 ?p dbr:Mathematician.}
    }
    ?o1 a dbo:Person;
      dbp:birthDate | dbo:birthDate ?subjectBirthDate;
      dbo:birthPlace ?birthPlace.
?birthPlace georss:point ?birthPoint;
     a  dbo:Settlement.
{
        {?o1 dbo:almaMater ?organisation.
     BIND('almaMater' AS ?r_property) }
UNION  {?o1 dbo:institution ?organisation.
     BIND('institution' AS ?r_property) }
}
?organisation  georss:point ?orgPoint.

    BIND(xsd:integer(SUBSTR(STR(?subjectBirthDate), 1, 4)) AS ?birthYear)
    FILTER ( (?birthYear >= 1451   ) )
 
          }
  ORDER BY ?birthYear
  }

```

## Inspecter les données

In [ ]:
### Ouvrir le fichier et créer un DataFrame
fd = 'data/dbpedia_sparql_reseaux_spatialisation.csv'
df = pd.read_csv(fd)
len(df), df.head(2)

In [ ]:
### Raccourcir l'URI, extraire l'année de naissance
df['subject_uri'] = df['subject_uri'].apply(lambda x : x.replace('http://dbpedia.org/resource/', ''))
df['organisation'] = df['organisation'].apply(lambda x : x.replace('http://dbpedia.org/resource/', ''))
df['birthPlace'] = df['birthPlace'].apply(lambda x : x.replace('http://dbpedia.org/resource/', ''))
df['BirthYear'] = df['subjectBirthDate'].apply(lambda x : int(x[:4]))

In [ ]:
### Séparer longitude et latitude, arrondir
df['latitudeBirth'] = df['birthPoint'].apply(lambda x : round(float((x.split(' ')[0])), 4))
df['longitudeBirth'] = df['birthPoint'].apply(lambda x : round(float((x.split(' ')[1])), 4))
df['latitudeOrg'] = df['orgPoint'].apply(lambda x : round(float((x.split(' ')[0])), 4))
df['longitudeOrg'] = df['orgPoint'].apply(lambda x : round(float((x.split(' ')[1])), 4))
df.head(2)

In [ ]:
### Inspecter le DataFrame
df.info()

In [ ]:
### Supprimer les colonnes inutiles
df = df.drop(['subjectBirthDate','birthPoint', 'orgPoint'], axis=1)

In [ ]:
### Inspecter les données
df.describe()

In [ ]:
### Inspecter les effectifs des propriétés
df.groupby(by='r_property').size()

In [ ]:
df.columns

## Un premier affichage exploratoire

On utilise la librairie GeoPandas qui apporte de nombreuses fonctionnalités SIG

In [ ]:
### Créer un dataframe geopandas avec une colonne contenant une géométrie 
# https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
birth_gdf = gpd.GeoDataFrame(
    df[['subject_uri', 'BirthYear', 'birthPlace','latitudeBirth', 'longitudeBirth']],\
    geometry=gpd.points_from_xy(df.longitudeBirth, df.latitudeBirth), crs="EPSG:4326"
)

birth_gdf.head(3)



In [ ]:
### Datasets Geopandas pour la couche de base
gpd.datasets.available

FutureWarning:

The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.

In [ ]:
#world = gpd.read_file(get_path("naturalearth.land"))
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
world.head()

In [ ]:
### Projection de la géographie
world.crs

In [ ]:
### Cartographier de toutes les naissances de personnes


# If we want to restrict to South America.
# world.clip([-90, -55, -25, 15])

ax = world.plot(color="white", edgecolor="black", figsize=(40,30))

# We can now plot our ``GeoDataFrame``.
birth_gdf.plot(ax=ax, color="red")

plt.show()

## Préparer et nettoyer les données


### Les lieux de naissance

In [ ]:
df[['birthPlace', 'latitudeBirth', 'longitudeBirth']].head(2)

In [ ]:
### Regrouper par lieu de naissance
df_lieux = pd.DataFrame(df.groupby(by=['birthPlace', 'latitudeBirth', 'longitudeBirth']).size())
df_lieux.reset_index(inplace = True)
df_lieux.columns=('birthPlace', 'latitude', 'longitude', 'effectif')
df_lieux.sort_values(by='effectif', ascending=False).head()

### Les géolocalisations des organisations

In [ ]:
df[['organisation','latitudeOrg', 'longitudeOrg']].head(2)

In [ ]:
### Regrouper par siège d'organisation
df_org = pd.DataFrame(df.groupby(by=['organisation','latitudeOrg', 'longitudeOrg']).size())
df_org.reset_index(inplace = True)
df_org.columns=('organisation', 'latitude', 'longitude', 'effectif')
df_org.sort_values(by='effectif', ascending=False).head()

### Traiter les doublons de personnes

In [ ]:
### Regrouper et compter par personne, lieu de naissance, organisation

person = pd.DataFrame(df.groupby(by=['subject_uri', 'BirthYear', 'birthPlace', 'r_property', 'organisation']).size())
person.reset_index(inplace=True)
person.columns = ("subject_uri","BirthYear","birthPlace", "r_property","organisation", "effectif")
#person.head()
show(person.sort_values(by="effectif",ascending=False))

In [ ]:
person.groupby(by=["subject_uri"]).size().sort_values(ascending=False).head(8)

In [ ]:
df.loc[df.subject_uri=="Franklin_E._Roach"]

In [ ]:
pp = pd.DataFrame(df.groupby(by=['subject_uri', 'BirthYear','r_property', 'organisation']).apply(lambda x : max(x.birthPlace)))
pp.reset_index(inplace=True)
pp.columns = ("subject_uri","birthYear","r_property", "organisation","birthPlace")

pp.head()

In [ ]:
pp.groupby(by=["subject_uri","birthPlace"]).size().sort_values(ascending=False).head(8)

In [ ]:
pp.loc[pp.subject_uri=="Paul_Erdős"]

In [ ]:
### On remarque qu'il n'y a plus de doublons à cause des lieux de naissance
# que des valeur uniques pour ces trois éléments
# La table pp contient donc les données nettoyées
pp.groupby(by=["subject_uri","birthYear","r_property", "organisation"]).size().sort_values(ascending=False).head(8)

### Ajouter les périodes

In [ ]:
min(pp.birthYear), max(pp.birthYear)

In [ ]:
### Violinplot : densité des naissances dans le temps
#  https://mode.com/blog/violin-plot-examples/
#  https://seaborn.pydata.org/generated/seaborn.violinplot.html#seaborn.violinplot

sns.set_theme(style="whitegrid")

sns.set(rc={"figure.figsize":(14, 7)})

a = pp.birthYear
lim_left=min(a)
lim_right=max(a)

ax = sns.violinplot(x=a)

### Noter que au delà des limites les valeurs sont coupées car postulées
ax.set_xlim(left=lim_left, right=lim_right)

In [ ]:
### Choisir des points dans le temps
# en fonction de la problématique ou heuristique
cuts = [1451, 1801, 1851, 1901, 1951, 2001]

In [ ]:
pp['cut'] = pd.cut(pp['birthYear'], bins=cuts, right=False )#, labels=cuts[:-1])
pp.head(2)

In [ ]:
pp['cut'] = pp['cut'].apply(lambda x : str(int(x.left))+'-'+ str(int(x.right)-1))
pp['cut'][:2]

In [ ]:
### Distribution des naissances par période
pd.DataFrame(pp.groupby(by=['cut']).size().reset_index(name='effectif'))

## Cartographier les naissances par périodes

In [ ]:
birth_period = pd.DataFrame(pp.groupby(by=['birthPlace', 'cut']).size().reset_index(name='effectif'))
birth_period.columns=['birthPlace', 'year_begin', 'effectif']
birth_period.head()

In [ ]:
### Ajouter les coordonnées des lieux de naissance 
bp_loc = pd.merge(birth_period, df_lieux, how='inner', left_on='birthPlace', 
                                     right_on='birthPlace', suffixes=('_periode', '_total')
              )
bp_loc.head()

In [ ]:
# now just plot it on a map with evolution by time
# https://plotly.com/python/animations/
# https://plotly.com/python/scatter-plots-on-maps/

# np.log(s) * 1000
size = [s if s != 0 else 0 for s in bp_loc.effectif_periode]

fig = px.scatter_geo(
    bp_loc,
    lat="latitude",
    lon="longitude",
    size=size,
    hover_name = "birthPlace",
    animation_frame="year_begin",
    width=1400, height=600,
    color_discrete_sequence=['red'],
    title="Evolution des lieux de naissance"
).update_layout(
    #mapbox_style='stamen-watercolor',
    #mapbox={"style": "carto-positron", "zoom":10},
    margin={"l": 0, "r": 20, "t": 30, "b": 0}

)

### Enregistrer l'image, puis l'ouvrir dans un autre onglet du navigateur
f_address = "pictures/birth_places.html"
fig.write_html(f_address)
fig.show()

### Grouper et compter les naissances par État contemporain

In [ ]:
### Polygones des États contemporains
world.head()

In [ ]:
## Projection des polygones
world.crs

In [ ]:
bp_loc.head()

In [ ]:
### Transformer les coordonnées géogr. en points
bp_loc_gdf = gpd.GeoDataFrame(
    bp_loc[['birthPlace', 'effectif_periode', 'year_begin']],\
    geometry=gpd.points_from_xy(bp_loc.longitude, bp_loc.latitude), crs="EPSG:4326"
)

bp_loc_gdf.head(3)


In [ ]:
### Vérification
bp_loc_gdf.loc[bp_loc_gdf['birthPlace']=='Fresno,_California']

In [ ]:
### Projection des points :
# doit être la même que celle des polygones 
bp_loc_gdf.crs

In [ ]:
### Jointure spatiale, par défault inner join
# vérifie si un point se trouve dans un polygone 
# et ajoute le pays du polygone
bp_loc_join = bp_loc_gdf.sjoin(world)

In [ ]:
show(bp_loc_join.sort_values(by='year_begin'), classes="display", scrollY="200px", 
     scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

In [ ]:
### Vérification
bp_loc_join.loc[bp_loc_join['birthPlace']=='Fresno,_California']

In [ ]:
### Regrouper par pays et époque, additionner les effectifs
birth_country = pd.DataFrame(bp_loc_join[['name', 'year_begin', 'effectif_periode', 'iso_a3']].groupby(by=['name','year_begin']).sum('effectif_periode').reset_index())
birth_country.columns=['name', 'year_begin', 'somme_effectif']
birth_country.head(10)

In [ ]:
### Transformer en json
countries_polygons = world[['name', 'geometry']]
countries_polygons.index = countries_polygons['name']
countries_polygons = countries_polygons.drop(['name'], axis=1)
countries_polygons_json = json.loads(countries_polygons.to_json())
countries_polygons.head()

In [ ]:
### Le pays avec le plus de naissances par période
birth_country[birth_country.somme_effectif==max(birth_country.somme_effectif)]

In [ ]:
### Distribution des effectifs des naissances
birth_country.somme_effectif.describe()

In [ ]:
### Distribution dans le dernier quartile
birth_country[birth_country.somme_effectif>6].somme_effectif.describe()

In [ ]:
# Représenter la densité de naissances de mathématiciens par pays
# https://plotly.com/python/animations/
# https://plotly.com/python-api-reference/generated/plotly.express.choropleth_mapbox.html
# https://medium.com/@lucas_bromerchenkel/choropleth-maps-with-time-sliders-using-plotly-df6e19e5f90c

# np.log(s) * 1000
size = [s if s != 0 else 0 for s in bp_loc.effectif_periode]

fig = px.choropleth_mapbox(
    birth_country,
    geojson=countries_polygons_json,
    locations="name",
    color="somme_effectif",
    mapbox_style='white-bg',
    zoom=1,
    color_continuous_scale='blues',
    ### valeur 20 définie en fonction de la distribution par pays
    # expérimenter en changeant la valeur
    range_color=(1, 20),
    animation_frame="year_begin",
    width=1000, height=600,
    title="Evolution des lieux de naissance"
).update_layout(
    #mapbox={"style": "carto-positron", "zoom":10},
    margin={"l": 0, "r": 0, "t": 30, "b": 0}

)
### Enregistrer l'image, puis l'ouvrir dans un autre onglet du navigateur
f_address = "pictures/birth_places_choropleth.html"
fig.write_html(f_address)

fig.show()

## Cartographier les organisations par périodes

In [ ]:
org_period = pd.DataFrame(pp.groupby(by=['organisation', 'cut']).size().reset_index(name='effectif'))
org_period.columns=['organisation', 'year_begin', 'effectif']
org_period.head()

In [ ]:
### Ajouter les coordonnées des lieux de naissance 
org_loc = pd.merge(org_period, df_org, how='inner', left_on='organisation', 
                                     right_on='organisation', suffixes=('_periode', '_total')
              )
org_loc.head()

In [ ]:
# now just plot it on a map with evolution by time
# https://plotly.com/python/animations/


size = [s if s != 0 else 0 for s in org_loc.effectif_periode]

px.scatter_geo(
    org_loc,
    lat="latitude",
    lon="longitude",
    size=size,
    hover_name = "organisation",
    animation_frame="year_begin",
    width=1200, height=600,
    color_discrete_sequence=['red'], 
    title="Evolution des emplacements des organisations" 
).update_layout(
     mapbox={"style": "carto-positron", "zoom":11},
    margin={"l": 0, "r": 20, "t": 30, "b": 0}

)


### Grouper et compter les appartenance aux organisations par État contemporain

In [ ]:
org_loc.head()

In [ ]:
### Transformer les coordonnées géogr. en points
org_loc_gdf = gpd.GeoDataFrame(
    org_loc[['organisation', 'effectif_periode', 'year_begin']],\
    geometry=gpd.points_from_xy(org_loc.longitude, org_loc.latitude), crs="EPSG:4326"
)

org_loc_gdf.head(5)


In [ ]:
### Projection des points :
# doit être la même que celle des polygones 
org_loc_gdf.crs

In [ ]:
### Jointure spatiale, par défault inner join
# vérifie si un point se trouve dans un polygone 
# et ajoute le pays du polygone
org_loc_join = org_loc_gdf.sjoin(world)

In [ ]:
show(org_loc_join.sort_values(by='year_begin'), classes="display", scrollY="200px", 
     scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

In [ ]:
### Vérification
org_loc_join.loc[org_loc_join['organisation']=='Indiana_University']

In [ ]:
### Regrouper par pays et époque, additionner les effectifs
org_country = pd.DataFrame(org_loc_join[['name', 'year_begin', 'effectif_periode', 'iso_a3']].groupby(by=['name', 'year_begin']).sum('effectif_periode').reset_index())
org_country.columns=['name', 'year_begin', 'somme_effectif']
org_country.head(10)

In [ ]:
### Transformer en json

# countries_polygons_json créé ci-dessus

In [ ]:
### Le pays avec le plus d'organisations par période
org_country[org_country.somme_effectif==max(org_country.somme_effectif)]

In [ ]:
### Distribution des effectifs des appartenances aux organisations
org_country.somme_effectif.describe()

In [ ]:
### Distribution dans le dernier quartile
org_country[org_country.somme_effectif>6].somme_effectif.describe()

In [ ]:
# Représenter la densité de naissances de mathématiciens par pays
# https://plotly.com/python/animations/
# https://plotly.com/python-api-reference/generated/plotly.express.choropleth_mapbox.html
# https://medium.com/@lucas_bromerchenkel/choropleth-maps-with-time-sliders-using-plotly-df6e19e5f90c

# np.log(s) * 1000
size = [s if s != 0 else 0 for s in org_loc.effectif_periode]

px.choropleth_mapbox(
    org_country,
    geojson=countries_polygons_json,
    locations="name",
    color="somme_effectif",

    mapbox_style='white-bg',
    zoom=1,
    color_continuous_scale='reds',
    ### valeur 20 définie en fonction de la distribution par pays
    # expérimenter en changeant la valeur
    range_color=(1, 35),
    animation_frame="year_begin",
    width=1000, height=600,
    title="Evolution des lieux de naissance"
).update_layout(
    #mapbox={"style": "carto-positron", "zoom":10},
    margin={"l": 0, "r": 0, "t": 30, "b": 0}

)


## Créer les pairs d'organisations

In [ ]:
### Reprendre la liste nettoyée des personnes
pp.head()

In [ ]:
### Reprendre la liste des organisations spatialisées
org_loc_join.head()

In [ ]:
org_country = pd.DataFrame(org_loc_join.groupby(by=["organisation", "name"]).sum("effectif_periode"))
org_country = org_country.reset_index()
org_country = org_country.drop(columns=["index_right", "pop_est", "gdp_md_est"])
org_country.columns=["organisation", "name", "effectif_total"]
org_country.sort_values(by="effectif_total", ascending=False ).head()

In [ ]:
ppm = pd.merge(pp, org_country, left_on="organisation", right_on="organisation", suffixes=('_p', '_y'))
ppm.head()

In [ ]:
[p for p in ppm.values][:2]

In [ ]:
## Pairs d'organisations
pairs_o = []

for s1 in ppm.sort_values(by='organisation').values:
    for s2 in ppm.sort_values(by='organisation').values:
        if s1[3] < s2[3] \
            and s1[0] == s2[0]:
                pairs_o.append([s1[3],s2[3], s1[5], s1[6], s2[6]])
        else:
            pass

In [ ]:
len(pairs_o), pairs_o[:2]

In [ ]:
### Créér dataframe et compter
liens_org = pd.DataFrame(pairs_o)
liens_org.columns=('org_1', 'org_2', 'periode', 'etat_1', 'etat_2')
liens_org.head()

In [ ]:
liens_org_grp = pd.DataFrame(liens_org.groupby(by=['org_1', 'org_2', 'periode']).size())
liens_org_grp=liens_org_grp.reset_index()
liens_org_grp.columns=['org_1', 'org_2', 'periode', 'effectif']
liens_org_grp.sort_values(by='effectif', ascending=False).head()

In [ ]:
### Ajouter les coordonnées des lieux de naissance 
org_1_loc = pd.merge(liens_org_grp, df_org, how='inner', left_on='org_1', 
                                     right_on='organisation', suffixes=('_p', '_y')
              )
org_1_loc=org_1_loc.drop(columns=['organisation','effectif_y'])
org_1_loc.head()

In [ ]:
### Ajouter les coordonnées des lieux de naissance 
orgs_loc = pd.merge(org_1_loc, df_org, how='inner', left_on='org_2', 
                                     right_on='organisation', suffixes=('_1', '_2')
              )
orgs_loc=orgs_loc.drop(columns=['organisation','effectif'])
orgs_loc.head()

In [ ]:
orgs_loc['geometry'] = orgs_loc.apply(lambda x: LineString([(x['longitude_1'], x['latitude_1']) , (x['longitude_2'], x['latitude_2'])]), axis = 1)
orgs_loc.head()

In [ ]:
orgs_loc.columns

In [ ]:
orgs_gdf = gpd.GeoDataFrame(
    orgs_loc[['org_1', 'org_2','periode','effectif_p', 'geometry']]
)

orgs_gdf.head(3)

In [ ]:
orgs_gdf = orgs_gdf.set_geometry('geometry')
orgs_gdf = orgs_gdf.set_crs('EPSG:4326')
orgs_gdf.crs

In [ ]:
### Cartographier


# If we want to restrict to South America.
# world.clip([-90, -55, -25, 15])

ax = world.plot(color="white", edgecolor="black", figsize=(40,30))

# We can now plot our ``GeoDataFrame``.
orgs_gdf.plot(ax=ax, color="red")

plt.show()

In [ ]:
orgs_loc.head()

In [ ]:
orgs_loc.columns

In [ ]:
orgs_loc = orgs_loc.sort_values(by='periode')

In [ ]:
#
lats = []
for s in orgs_loc.values:
    lats.append(np.array(np.float64(s[4]), np.float64(s[6])))
lats[:3]

In [ ]:
#
longs = []
for s in orgs_loc.values:
    longs.append(np.array(np.float64(s[5]), np.float64(s[7])))
longs[:3]

In [ ]:
#
names = []
for s in orgs_loc.values:
    names.append(s[0]+ '_' + s[1])
names[:3]

In [ ]:
#
size = []
for s in orgs_loc.values:
    size.append(s[3])
size[:3]

In [ ]:
#
periodes = []
for s in orgs_loc.values:
    periodes.append(s[2])
periodes[:3]

In [ ]:
# now just plot it on a map with evolution by time
# https://plotly.com/python/animations/


size = [s if s != 0 else 0 for s in org_loc.effectif_periode]

size = [s if s != 0 else 0 for s in org_loc.effectif_periode]

"""px.scatter_geo(
    org_loc,
    lat="latitude",
    lon="longitude",
    size=size,
    hover_name = "organisation",
    animation_frame="year_begin",
    width=1200, height=600,
    color_discrete_sequence=['red'], 
    title="Evolution des emplacements des organisations" 
)"""

fig = px.line_geo(#data_frame=orgs_loc,
            lat=lats, lon=longs, 
       # hover_name=names,
    #size=size,
    animation_frame=periodes,
    width=1200, height=600,
    color_discrete_sequence=['red'], 
    title="Evolution des emplacements des organisations" 
).update_layout(
     mapbox={"style": "carto-positron", "zoom":11},
    margin={"l": 0, "r": 20, "t": 30, "b": 0}

)

### Enregistrer l'image, puis l'ouvrir dans un autre onglet du navigateur
f_address = "pictures/org_relationships.html"
fig.write_html(f_address)

fig.show()

### Observer plus finemenent

In [ ]:
liens_etat_grp = pd.DataFrame(liens_org.groupby(by=['etat_1', 'etat_2', 'periode']).size())
liens_etat_grp=liens_etat_grp.reset_index()
liens_etat_grp.columns=['etat_1', 'etat_2', 'periode', 'effectif']
liens_etat_grp.sort_values(by='effectif', ascending=False).head()

In [ ]:
liens_etat_grp['relation'] = liens_etat_grp.apply(lambda x : x.etat_1 + "-" + x.etat_2, axis=1)
liens_etat_grp.head()

In [ ]:
liste_relations = pd.DataFrame(liens_etat_grp.groupby(by='relation').sum('effectif'))
liste_relations = liste_relations.reset_index()
liste_relations.sort_values(by='effectif',ascending=False).head(10)

In [ ]:
l_rel = list(liste_relations.relation)
l_rel[:3]

In [ ]:
### Distribution des naissances par période
df_cut = pd.DataFrame(pp.groupby(by=['cut']).size().reset_index(name='effectif'))
df_cut

In [ ]:
v_cut = list(df_cut.cut)
v_cut

In [ ]:
toute_relation = []
for r in l_rel:
    for c in v_cut:
        toute_relation.append([r,c])
df_toute_rel = pd.DataFrame(toute_relation)
df_toute_rel.columns = ('relation', 'periode')
df_toute_rel.head(10)

In [ ]:
merge_relations = pd.merge(df_toute_rel, liens_etat_grp[['periode', 'effectif', 'relation']], how='left',  left_on=['relation', 'periode'], right_on=['relation', 'periode'])
merge_relations.head(10)

In [ ]:
merge_relations.effectif = merge_relations.effectif.fillna(0)
merge_relations.effectif= merge_relations.effectif.apply(lambda x : int(x))
merge_relations.head(10)

In [ ]:
file_addr = 'data/merge_relations.csv'
merge_relations.to_csv(file_addr)

In [ ]:
show(merge_relations)

In [ ]:
list(merge_relations.sort_values(by='periode').periode.unique())

In [ ]:
### Explorer par des filtres Pandas
liens_etat_grp[liens_etat_grp.periode=="1451-1800"].sort_values(by=["effectif"],ascending=False).head(10)

### Création et utilisation d'une fonction d'affichage

In [ ]:
# generate the plot

def generate_fig(df, country=None, load_image = False):
    
    if country:
        dataframe = df[df.relation.str.contains(country)]
    else:
        dataframe = df
    
    fig = px.bar(
        dataframe,
        x="effectif",
        y="relation",
        base="relation",
        #color="effectif",
        #orientation="h",
        height= len(dataframe)*5,
        width=1000,
        animation_frame="periode"

    )

    fig.update_layout(
        margin=dict(l=2, r=100, t=0, b=5),
        paper_bgcolor="LightSteelBlue",
        xaxis={"range":[0,list(merge_relations.effectif.quantile([0.99]))[0]]}
#                        dataframe.effectif.max()]}
    )
    
    ### enregistrer au format HTML et ouvrir comme page web
    if country:
        f_address = "pictures/countries_relations_" + country + ".html"
    else:
        f_address = "pictures/countries_relations.html"
    fig.write_html(f_address)

    ### On n'affiche pas l'image directement mais on la sauvegarde dans un fichier,
    # puis on affiche le fichier dans le carnet Jupyter avec l'instruction display.HMTL,
    # cf. ci-dessous. On peut aussi ouvrir le fichier dans une autre fenêtre
    
    if load_image == False:
        fig.show()
    else:
        i = IPython.display.HTML(filename=f_address)
        return i



In [ ]:
### inspecter le dataframe complet
df = merge_relations
generate_fig(df)

In [ ]:
### Filtrer sur le nom d'un pays
generate_fig(df, 'Italy', load_image=True)